# Get IDs from Videos

In [7]:
from googleapiclient.discovery import build

import os

In [9]:
youtube = build('youtube', 'v3', developerKey=os.environ["YOUTUBE_API_KEY"])

res =  youtube.channels().list(id='UC16niRr50-MSBwiO3YDb3RA', #Copy and paste here ID channel
                               part='contentDetails').execute()

In [10]:
## Get upload playlist
res

{'kind': 'youtube#channelListResponse',
 'etag': 'sZ-Jnqk906RCqYi5ZLpfqZ81N9M',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '3tLMxTHqqlpjJBE5lOA_XOLIkhQ',
   'id': 'UC16niRr50-MSBwiO3YDb3RA',
   'contentDetails': {'relatedPlaylists': {'likes': '',
     'uploads': 'UU16niRr50-MSBwiO3YDb3RA'}}}]}

# Extract comments and other variables

In [11]:
# Get the Playlist ID from res. Here the paylist is uploads. All the upload content.
playlist_ids = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']

In [12]:
playlist_ids = ['PL6XRrncXkMaUNhifXWtS3gZtctnZpUZPw']

In [13]:
def get_all_video_ids_from_playlists(youtube, playlist_ids):
    all_videos = []  # Initialize a single list to hold all video IDs

    for playlist_id in playlist_ids:
        next_page_token = None

        # Fetch videos from the current playlist
        while True:
            playlist_request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=playlist_id,
                pageToken=next_page_token)
            playlist_response = playlist_request.execute()

            for item in playlist_response['items']:
                video_id = item['contentDetails']['videoId']
                video_request = youtube.videos().list(
                    part='contentDetails',
                    id=video_id
                )
                video_response = video_request.execute()
                duration = video_response['items'][0]['contentDetails']['duration']

                # Check if the video is not a short (you can adjust the duration threshold as needed)
                if 'M' in duration or 'H' in duration:
                    all_videos.append(video_id)

            next_page_token = playlist_response.get('nextPageToken')

            if next_page_token is None:
                break

    return all_videos

In [14]:
video_ids = get_all_video_ids_from_playlists(youtube, playlist_ids)

In [15]:
video_ids

['QbpxLZGLMGs',
 'E-5Zf4D5jmI',
 'v_UZjVMcOkk',
 'JCAza8k6CyI',
 '5OTXiFtbpjQ',
 'djCKmQVkftY',
 'VNZgfKiv__o',
 '-VuuL_AYvVs',
 'uk-ZzeC96BM',
 '5F6YRQKZX9E',
 'y8Cg3LwIcZk',
 '65CI8hznDy4',
 'YC4CXDAsgTo',
 'AnCatG15hOM',
 'UyFSa_0ei48',
 'jHfrf_DGTGE',
 'vwvwpn0fays',
 'KQTKy64PuOE',
 'ZJShiSyRLVg',
 'HuKo31IFAt0',
 'urmDcQKB8eY',
 'q3nMcZRFHSs',
 'WeKjWxwy5jI',
 'x8losEBzCtU',
 'fWKeCfbPqv8',
 '8FjvJxxlFtI',
 'tJWTd8O1oKk',
 'frFnqQ9Jajg',
 'zApjeJtB6gI',
 'n9IrowRkGFE',
 'M_NsHTOL258',
 'QkxXpCiYI0U',
 'rBydMHT8tVU',
 'Fvziacme9XI',
 'm8qfD5nWSec',
 'BQaJX26wgnQ',
 'ri-Lvmfzf2s',
 '7w6c6oGs2_w',
 'FccwpafufPQ',
 'KSNs8P3MrMY',
 'R5MJz1-dMQ4',
 'hrG1VqDQhBQ',
 'zdCqSi0KH2c',
 'g0BlDeU8rlE',
 '9OrrLvXpmKo',
 'ZX_dFwCaX-4']

In [16]:
def get_all_video_ids_from_playlists(youtube, playlist_ids):
    all_videos = []  # Initialize a single list to hold all video IDs

    for playlist_id in playlist_ids:
        next_page_token = None

        # Fetch videos from the current playlist
        while True:
            playlist_request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=playlist_id,
                pageToken=next_page_token)
            playlist_response = playlist_request.execute()

            all_videos += [item['contentDetails']['videoId'] for item in playlist_response['items']]

            next_page_token = playlist_response.get('nextPageToken')

            if next_page_token is None:
                break

    return all_videos

# Fetch all video IDs from the specified playlists
video_ids = get_all_video_ids_from_playlists(youtube, playlist_ids)

# Now we can pass video_ids to the next function
# next_function(video_ids)

In [17]:
len(video_ids)

47

## Get the comments of videos

In [ ]:
import pandas as pd
from googleapiclient.errors import HttpError

box = [['Video Title', 'Name', 'Comment', 'Time', 'Likes', 'Reply Count']]

def scrape_comments_with_replies(video_id):
    try:
        video_info = youtube.videos().list(part='snippet', id=video_id).execute()
        video_title = video_info['items'][0]['snippet']['title']
    except HttpError as e:
        if e.resp.status == 403:
            print(f"Comments are disabled for video {video_id}. Skipping...")
            return None
        else:
            print(f"An error occurred while fetching video info for video {video_id}: {e}")
            return None

    try:
        data = youtube.commentThreads().list(part='snippet', videoId=video_id, maxResults='100', textFormat="plainText").execute()
    except HttpError as e:
        if e.resp.status == 403:
            print(f"Comments are disabled for video {video_id}. Skipping...")
            return None
        else:
            print(f"An error occurred while fetching comments for video {video_id}: {e}")
            return None

    for i in data["items"]:
        name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
        comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
        published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
        likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
        replies = i["snippet"]['totalReplyCount']

        box.append([video_title, name, comment, published_at, likes, replies])

        totalReplyCount = i["snippet"]['totalReplyCount']

        if totalReplyCount > 0:
            parent = i["snippet"]['topLevelComment']["id"]

            try:
                data2 = youtube.comments().list(part='snippet', maxResults='100', parentId=parent, textFormat="plainText").execute()
            except HttpError as e:
                print(f"An error occurred while fetching replies for comment {parent} of video {video_id}: {e}")
                continue

            for i in data2["items"]:
                name = i["snippet"]["authorDisplayName"]
                comment = i["snippet"]["textDisplay"]
                published_at = i["snippet"]['publishedAt']
                likes = i["snippet"]['likeCount']
                replies = ''

                box.append([video_title, name, comment, published_at, likes, replies])

    while ("nextPageToken" in data):
        try:
            data = youtube.commentThreads().list(part='snippet', videoId=video_id, pageToken=data["nextPageToken"], maxResults='100', textFormat="plainText").execute()
        except HttpError as e:
            print(f"An error occurred while fetching next page of comments for video {video_id}: {e}")
            break

        for i in data["items"]:
            name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
            comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
            published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
            likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
            replies = i["snippet"]['totalReplyCount']

            box.append([video_title, name, comment, published_at, likes, replies])

            totalReplyCount = i["snippet"]['totalReplyCount']

            if totalReplyCount > 0:
                parent = i["snippet"]['topLevelComment']["id"]

                try:
                    data2 = youtube.comments().list(part='snippet', maxResults='100', parentId=parent, textFormat="plainText").execute()
                except HttpError as e:
                    print(f"An error occurred while fetching replies for comment {parent} of video {video_id}: {e}")
                    continue

                for i in data2["items"]:
                    name = i["snippet"]["authorDisplayName"]
                    comment = i["snippet"]["textDisplay"]
                    published_at = i["snippet"]['publishedAt']
                    likes = i["snippet"]['likeCount']
                    replies = ''

                    box.append([video_title, name, comment, published_at, likes, replies])

    df = pd.DataFrame({'Video Title': [i[0] for i in box], 'Name': [i[1] for i in box], 'Comment': [i[2] for i in box],
                       'Time': [i[3] for i in box], 'Likes': [i[4] for i in box], 'Reply Count': [i[5] for i in box]})

    return df

# Iterate over video IDs and save comments in a single file
all_comments = pd.DataFrame()
for video_id in video_ids[0:1]:
    comments_df = scrape_comments_with_replies(video_id)
    if comments_df is not None:
        all_comments = pd.concat([all_comments, comments_df], ignore_index=True)


In [27]:

# Save the comments
all_comments.to_csv('../data/test_youtube_comments.csv', index=False)

print("Successful! Check the CSV file that you have just created.")


Successful! Check the CSV file that you have just created.
